In [17]:
from PIL import Image
import numpy as np

def load_image(path: str):
    try:
        return np.asarray(Image.open(path).convert('L'))
    except:
        print('Error loading image')

from line_calculator import BresenhamLineCalculator
from source_detector_pair_generator import SourceDetectorPairGenerator
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt

def radon_transform(source_detector_pair_generator, iterations, image):
    sinogram = []
    for i in range(iterations):
        iteration_result = []
        for pair in source_detector_pair_generator.generate(i):
            detector_result = 0
            for pixel in BresenhamLineCalculator.calculate_between(pair.source, pair.detector):
                detector_result += image[pixel.x - 1][pixel.y - 1]
            detector_result /= 255
            iteration_result.append(detector_result)
        sinogram.append(iteration_result)
    return sinogram


def ramp_filter(ffts):
    ffts = np.asarray(ffts)
    ramp = np.floor(np.arange(0.5, ffts.shape[1]//2 + 0.1, 0.5))
    return ffts * ramp

def reverse_radon_transform(image_height, spread, sinogram):
    image = np.zeros((image_height, image_height))
    source_detector_pair_generator = SourceDetectorPairGenerator(
        image_height,
        len(sinogram[0]),
        spread,
        360//len(sinogram)
    )
    
    for i in range(len(sinogram) - 1):
        for pair_index, pair in enumerate(source_detector_pair_generator.generate(i)):
            for pixel in BresenhamLineCalculator.calculate_between(pair.source, pair.detector):
                image[pixel.x - 1][pixel.y - 1] += sinogram[i][pair_index] / 255
    
    return image
            
    

def main(image_path: str, detector_count: int, spread: int, step: int):
    image = load_image(image_path)
    diameter = len(image) if len(image) <= len(image[0]) else len(image[0])
    
    plt.figure()
    plt.imshow(image, cmap='gray')
    
    iterations = 360 // step
    source_detector_pair_generator = SourceDetectorPairGenerator(diameter, detector_count, spread, step)
    sinogram = radon_transform(source_detector_pair_generator, iterations, image)
    
    plt.figure()
    plt.imshow(sinogram, cmap='gray')

    result = reverse_radon_transform(diameter, spread, sinogram)
    
    plt.figure()
    plt.imshow(result, cmap='gray')


interact(
    main,
    image_path='/Users/fortek/Downloads/tomograf-zdjecia/Kropka.jpg',
    detector_count=widgets.IntSlider(min=0, max=300, step=1, value=180, continuous_update=False),
    spread=widgets.IntSlider(min=0, max=180, step=1, value=80, continuous_update=False),
    step=widgets.IntSlider(min=1, max=360, step=1, value=4, continuous_update=False)
)

interactive(children=(Text(value='/Users/fortek/Downloads/tomograf-zdjecia/Kropka.jpg', description='image_pat…

<function __main__.main(image_path: str, detector_count: int, spread: int, step: int)>